# Imports

In [ ]:
# Import necessary libraries for mathematical operations and plotting
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.patches import Rectangle

# Import astropy modules for working with astronomical data
import astropy.units as u
from astropy.visualization import simple_norm, SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.wcs import WCS
from astropy.wcs.utils import proj_plane_pixel_scales
from astropy.coordinates import angular_separation, Angle, SkyCoord
from astropy.stats import sigma_clipped_stats
from astropy.io import fits
from astropy.nddata import Cutout2D
# Import os for directory operations
import os
# Import astroalign for aligning astronomical images
import astroalign as align
# Import twirl for peak finding and WCS (World Coordinate System) computations
from twirl import find_peaks
from twirl import gaia_radecs
from twirl.geometry import sparsify
from twirl import compute_wcs
# Import photutils for aperture photometry
from photutils.aperture import SkyCircularAperture
from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture
from pylab import figure, cm
# Import pandas for data manipulation
import pandas as pd

import random

In [ ]:
#Constnats
# BLUE_FILTER_COLOR =  'Blues'
# RED_FILTER_COLOR =  'Reds'

BLUE_FILTER_COLOR =  cm.grey
RED_FILTER_COLOR =  cm.grey

# Helper funcitons

In [ ]:
def get_filename(directory):
    '''
    This gives all the filenames that are in a folder and returns them in list format
    '''
    filename_list=[]
    for filename in os.scandir(directory):
        if os.path.isfile(filename):
            filename_list.append(filename)
    return(filename_list)

def stack_frames(file_list):
    '''
    This takes a list of filenames
    Loads the data, averages all the images together
    Returns a master image
    '''
    imgs1=fits.open(file_list[0])
    data=imgs1[0].data

    height=len(data[:,0])
    width=len(data[0,:])
    
    master_img=np.zeros((height, width))
    count=0
    for file in file_list:
        count+=1
        imgs=fits.open(file)
        img_data=imgs[0].data
        master_img=master_img+img_data

    master_img=master_img/count

    return master_img

def correct_lights(light, dark, norm_flats_bias):
    '''
    light = list of light file names
    dark = master dark image
    norm_flats_bias = normalized bias subtracted flat images

    returns - list of image HDUs
    '''
    corr_HDU_list=[]
    for file in light:
        light_imgs = fits.open(file)
        light_data = light_imgs[0].data
        light_corr = (light_data - dark)/norm_flats_bias
        corr_HDU_list.append(light_corr)
    return corr_HDU_list

def align_image(list_of_images, reference_image):
    '''
    under costruction
    '''

    height=len(reference_image[:,0])
    width=len(reference_image[0,:])
    Master_image=np.zeros((height, width))
    count=0
    for image in list_of_images:
        count+=1
        aligned_image, footprint=align.register(image, reference_image)
        #after align, now stack em up
        Master_image=Master_image+aligned_image

    Master_image=Master_image/count

    return Master_image

# Target

In [ ]:
DIRECTORY="m2/"
# DIRECTORY="m13/"

# Gather all the files

In [ ]:
#Get the file directory
darks_directory     = DIRECTORY + "darks/120/"
bias_directory      = DIRECTORY + "bias/2x2/"

flats1B_directory   = DIRECTORY + "Night 1/flats/B"
flats1R_directory   = DIRECTORY + "Night 1/flats/R"
flats2B_directory   = DIRECTORY + "Night 2/flats/B"
flats2R_directory   = DIRECTORY + "Night 2/flats/R"

light1B_directory   = DIRECTORY + "Night 1/B"
light1R_directory   = DIRECTORY + "Night 1/R"
light2B_directory   = DIRECTORY + "Night 2/B"
light2R_directory   = DIRECTORY + "Night 2/R"

#Create arays of file names, it's easier to reference them
darks_files         = get_filename(darks_directory)
bias_files          = get_filename(bias_directory)

flats1B_files       = get_filename(flats1B_directory)
flats1R_files       = get_filename(flats1R_directory)
flats2B_files       = get_filename(flats2B_directory)
flats2R_files       = get_filename(flats2R_directory)

light1B_files       = get_filename(light1B_directory)
light1R_files       = get_filename(light1R_directory)
light2B_files       = get_filename(light2B_directory)
light2R_files       = get_filename(light2R_directory)

print("Detected " + str(len(darks_directory)) + " dark images")
print("Detected " + str(len(bias_files)) + " bias images")
print("Detected " + str(len(flats1B_files)) + " Blue Filter images from Nov 2nd 2024")
print("Detected " + str(len(flats1R_files)) + " Red Filter images from Nov 2nd 2024")
print("Detected " + str(len(flats2B_files)) + " Blue Filter images from Nov 5th 2024")
print("Detected " + str(len(flats2R_files)) + " Red Filter images from Nov 5th 2024")

# Make master bias and darks

In [ ]:
master_bias = stack_frames(bias_files) 
master_dark = stack_frames(darks_files)

# Make master flats

In [ ]:
#stack flat images for each day & filter to creat 4 master flats
flat1B_img = stack_frames(flats1B_files)
flat1R_img = stack_frames(flats1R_files)
flat2B_img = stack_frames(flats2B_files)
flat2R_img = stack_frames(flats2R_files)

#Subtract the bias from the flats
flat1B_bias = flat1B_img - master_bias 
flat1R_bias = flat1R_img - master_bias
flat2B_bias = flat2B_img - master_bias
flat2R_bias = flat2R_img - master_bias

#Normalize by dividing by the maximum value of the array
master_flat_1B = flat1B_bias / np.max(flat1B_bias) 
master_flat_1R = flat1R_bias / np.max(flat1R_bias)
master_flat_2B = flat2B_bias / np.max(flat2B_bias)
master_flat_2R = flat2R_bias / np.max(flat2R_bias)


# Correct Lights

In [ ]:
master_light_1B = correct_lights(light1B_files, master_dark, master_flat_1B)
master_light_1R = correct_lights(light1R_files, master_dark, master_flat_1R)
master_light_2B = correct_lights(light2B_files, master_dark, master_flat_2B)
master_light_2R = correct_lights(light2R_files, master_dark, master_flat_2R)

# Choose a reference image

In [ ]:
# grab the first night
ref_img_for_b_filter = fits.open(light1B_files[0])
ref_img_for_r_filter = fits.open(light1R_files[0])

# Get the data off the reference image

In [ ]:
#get the header and the actual image from the choosen reference image
ref_header_b_filter   = ref_img_for_b_filter[0].header # this header will be stamped on any fits I make
ref_header_r_filter   = ref_img_for_r_filter[0].header # this header will be stamped on any fits I make

ref_data_b_filter     = ref_img_for_b_filter[0].data   # this is be use for allighining light frames
ref_data_r_filter     = ref_img_for_r_filter[0].data   # this is be use for allighining light frames

# Alighn light frame from each filters

In [ ]:
# make a list of all the B and R filter of the lights 
b_filter_lights = master_light_1B + master_light_2B
r_filter_lights = master_light_1R + master_light_2R

# alligh the lights using that random image we picked
master_Bfilter_light = align_image(b_filter_lights, ref_data_b_filter)
master_Rfilter_light = align_image(r_filter_lights, ref_data_r_filter)

# create the fits file to work with
b_filter_fits = fits.PrimaryHDU(master_Bfilter_light, ref_header_b_filter)
r_filter_fits = fits.PrimaryHDU(master_Rfilter_light, ref_header_r_filter)

# Let's take a look at what we have so far

In [ ]:
# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Plot the B filter image in the first subplot
ax1.imshow(b_filter_fits.data, cmap = BLUE_FILTER_COLOR, norm = LogNorm(vmin = 1, vmax = 100000))
ax1.set_title('B Filter')

# Plot the R filter image in the second subplot
ax2.imshow(r_filter_fits.data, cmap = RED_FILTER_COLOR, norm = LogNorm(vmin = 1, vmax = 100000))
ax2.set_title('R Filter')

# Add colorbars to both subplots
fig.colorbar(ax1.imshow(b_filter_fits.data, cmap = BLUE_FILTER_COLOR, norm = LogNorm( vmin = 1, vmax = 100000 )), ax = ax1)
fig.colorbar(ax2.imshow(r_filter_fits.data, cmap = RED_FILTER_COLOR, norm = LogNorm( vmin = 1, vmax = 100000 )), ax = ax2)

# Adjust the layout to prevent overlap
plt.tight_layout()

# Save the figure to the drive
output_filepath = DIRECTORY + "B & R filter lights.jpg"
plt.savefig(output_filepath, format='jpg')

# Display the plot
plt.show()
plt.clf()

# Save the Lights as FITS files

In [ ]:
# locate the place you wanna save the fits files
output_directory_b_filter = DIRECTORY + "Light_B.fits"
output_directory_r_filter = DIRECTORY + "Light_R.fits"

# write the files to drive
b_filter_fits.writeto(output_directory_b_filter, overwrite = True)
r_filter_fits.writeto(output_directory_r_filter, overwrite = True)
